In [2]:
import pandas as pd
df = pd.read_pickle('golfDB.pkl')

In [12]:
df.head()

,id,youtube_id,player,sex,club,view,slow,events,bbox,split
0,0,f1BWA5F87Jc,SANDRA GAL,f,driver,down-the-line,0,"[408, 455, 473, 476, 490, 495, 498, 501, 514, ...","[0.09765625000000001, 0.006944444444444444, 0....",3
1,1,f1BWA5F87Jc,SANDRA GAL,f,driver,down-the-line,1,"[814, 854, 917, 931, 988, 1006, 1019, 1030, 10...","[0.039062500000000014, 0.0006944444444444445, ...",3
2,2,tA1iotgtMyc,CHRIS DIMARCO,m,driver,down-the-line,0,"[521, 659, 678, 683, 692, 696, 698, 701, 715, ...","[0.165625, 0.0006944444444444445, 0.48359375, ...",3
3,3,tA1iotgtMyc,CHRIS DIMARCO,m,driver,down-the-line,1,"[1106, 1190, 1244, 1264, 1300, 1313, 1324, 133...","[0.18515625, 0.0006944444444444445, 0.465625, ...",3
4,4,wDCKLePrwHA,BROOKE HENDERSON,f,driver,down-the-line,0,"[157, 170, 183, 188, 197, 201, 205, 207, 220, ...","[0.11015625, 0.0006944444444444445, 0.4984375,...",3


In [2]:
!pip install pytubefix

In [3]:
import os
import signal
from pytubefix import YouTube
from tqdm import tqdm
from functools import partial

def timeout_handler(signum, frame):
    raise TimeoutError("Download timed out")

def download_youtube_video(youtube_id, output_path, timeout=20):  # 5 minutes timeout by default
    url = f'https://youtu.be/{youtube_id}'
    try:
        # Set up the timeout
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(timeout)

        yt = YouTube(url)
        stream = yt.streams.first()
        output_file = os.path.join(output_path, f'{youtube_id}.mp4')
        stream.download(output_path=output_path, filename=f'{youtube_id}.mp4')

        # Cancel the alarm if download completes before timeout
        signal.alarm(0)

        if os.path.exists(output_file):
            return True
        else:
            print(f"File not saved for {youtube_id}")
            return False
    except TimeoutError:
        print(f"Download timed out for {youtube_id}")
        return False
    except Exception as e:
        print(f"Error downloading {youtube_id}: {str(e)}")
        return False

# Create a directory to store the downloaded videos
output_directory = 'downloaded_videos'
os.makedirs(output_directory, exist_ok=True)

# Get unique youtube_ids
unique_youtube_ids = df['youtube_id'].unique()

# Download videos for each unique youtube_id with progress bar
successful_downloads = 0
total_videos = len(unique_youtube_ids)

for youtube_id in tqdm(unique_youtube_ids, desc="Downloading videos", unit="video"):
    if download_youtube_video(youtube_id, output_directory):
        successful_downloads += 1
    else:
        print(f"Failed to download video: {youtube_id}")

print(f"Download complete. Successfully downloaded {successful_downloads} out of {total_videos} videos.")
print(f"Videos should be saved in: {os.path.abspath(output_directory)}")


KeyboardInterrupt: 

In [5]:
df[(df['player'] == 'TIGER WOODS') & (df['club'] == 'iron') ]['bbox'][21]

array([9.06250000e-02, 6.94444444e-04, 7.04687500e-01, 1.00000000e+00])

In [5]:
import os
import cv2
# Get all file names in downloaded_videos folder
downloaded_files = os.listdir('downloaded_videos')
rows_with_downloaded_videos = [x.split('.')[0] for x in downloaded_files]

# Add a new column called processed_video_path
df['processed_video_path'] = None

for idx, row in df.iterrows():
    if idx != 856 and idx != 857 and idx != 858 and idx != 855:
        if row['youtube_id'] in rows_with_downloaded_videos:
            video_path = os.path.join('downloaded_videos', f"{row['youtube_id']}.mp4")
            if os.path.exists(video_path):
                cap = cv2.VideoCapture(video_path)
                frame_count = 0
                new_frames = []
                while cap.isOpened():
                    ret, frame = cap.read()
                    if not ret:
                        break
                    if frame_count + 1 >= int(row['events'][1]) and frame_count + 1 <= int(row['events'][-1]):
                        x1, y1, x2, y2 = row['bbox']
                        height, width, _ = frame.shape
                        x1 = int(x1 * width)
                        y1 = int(y1 * height)
                        y2 = int(y2 * height)
                        x2 = x1 + (y2 - y1)
                        cropped_frame = frame[y1:y2, x1:x2]
                        new_frames.append(cropped_frame)
                    
                    # Process the frame here
                    frame_count += 1
                if new_frames:
                    try:
                        import uuid
                        random_id = uuid.uuid4()
                        output_video_path = os.path.join('processed_videos', f"{row['player'].replace(' ', '_')}_{row['youtube_id']}_{random_id}.mp4")
                        height, width, layers = new_frames[0].shape
                        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
                        out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (width, height))

                        for frame in new_frames:
                            out.write(frame)
                        df.at[idx, 'processed_video_path'] = output_video_path
                    except Exception as e:
                        print(f"Error processing video {row['youtube_id']}: {str(e)}")
                    out.release()
                cap.release()
                print(idx)
            else:
                print(f"Video file {video_path} does not exist")
    
    # Save the CSV after processing each video
    df.to_csv('golfDB_processed.csv', index=False)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134


[mov,mp4,m4a,3gp,3g2,mj2 @ 0x651edd5a87c0] stream 0, offset 0xa818: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x651edd5a87c0] stream 0, offset 0xb323: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x651edd5a87c0] stream 0, offset 0xa818: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x651edd5a87c0] stream 0, offset 0xb323: partial file


135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384


[mov,mp4,m4a,3gp,3g2,mj2 @ 0x651ee2524500] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x651ee2245d40] moov atom not found


1198
1199
1200
1201
1202
1203
1204
1205
1206
1207
1208
1209
1210
1211
1212
1213
1214
1215
1216
1217
1218
1219
1220
1221
1222
1223
1224
1225
1226
1227
1228
1229
1230
1231
1232
1233
1234
1235
1236
1237
1238
1239
1240
1241
1242
1243
1244
1245
1246
1247
1248
1249
1250
1251
1252
1253
1254
1255
1256
1257
1258
1259
1260
1261
1262
1263
1264
1265
1266
1267
1268
1269
1270
1271
1272
1273
1274
1275
1276
1277
1278
1279
1280
1281
1282
1283
1284
1285
1286
1287
1288
1289
1290
1291
1292
1293
1294
1295
1296
1297
1298
1299
1300
1301
1302
1303
1304
1305
1306
1307
1308
1309
1310
1311
1312
1313
1314
1315
1316
1317
1318
1319
1320
1321
1322
1323
1324
1325
1326
1327
1328
1329
1330
1331
1332
1333
1334
1335
1336
1337
1338
1339
1340
1341
1342
1343
1344
1345
1346
1347
1348
1349
1350
1351
1352
1353
1354
1355
1356
1357
1358
1359
1360
1361
1362
1363
1364
1365
1366
1367
1368
1369
1370
1371
1372
1373
1374
1375
1376
1377
1378
1379
1380
1381
1382
1383
1384
1385
1386
1387
1388
1389
1390
1391
1392
1393
1394
1395
1396
1397
